In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
!which python

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

from retDNN import *

import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

In [4]:
df = pd.read_csv("Eco1_ncRNAs_with_scores_processed.tsv", sep="\t", header=0, index_col=0)
df.head()

,synthesis,Sublibrary,fasta,prefix,"Score (enrichment/depletion relative to mean wt, wt=100)",SD,N,ncRNA,score,length,scoreINT
item_1,r1,L7,4235,86_r1_L7,1.219663,0.544209,3,TGCGCACCCTTAGCGAGAGGTTTATCATTAAGGTCAACCTCTGGAT...,0.287665,119,-1.205478
item_2,r1,L7,4236,86_r1_L7,0.244852,0.214515,3,TGCGCACCCTTAGCGAGAGGTTTATCATTAAGGTCAACCTCTGGAT...,-2.024137,121,-1.400132
item_3,r1,L7,4237,86_r1_L7,1.884223,1.060321,3,TGCGCACCCTTAGCGAGAGGTTTATCATTAAGGTCAACCTCTGGAT...,0.914735,123,-1.120598
item_4,r1,L7,4238,86_r1_L7,6.821973,3.187198,3,TGCGCACCCTTAGCGAGAGGTTTATCATTAAGGTCAACCTCTGGAT...,2.770400,125,-0.942509
item_5,r1,L7,4239,86_r1_L7,6.150030,2.305129,3,TGCGCACCCTTAGCGAGAGGTTTATCATTAAGGTCAACCTCTGGAT...,2.620828,127,-0.950524


In [5]:
seq_dict = dict(zip(df.index,df.ncRNA))
score_dict = dict(zip(df.index,df.scoreINT))

In [19]:
df_train = pd.read_csv('data/train.csv', index_col=0, header=0)
df_valid = pd.read_csv('data/valid.csv', index_col=0, header=0)
df_test = pd.read_csv('data/test.csv', index_col=0, header=0)
df_test.head()

,synthesis,Sublibrary,fasta,prefix,"Score (enrichment/depletion relative to mean wt, wt=100)",SD,N,ncRNA,score,length,scoreINT
item_2,r1,L7,4236,86_r1_L7,0.244852,0.214515,3,TGCGCACCCTTAGCGAGAGGTTTATCATTAAGGTCAACCTCTGGAT...,-2.024137,121,-1.400132
item_11,r1,L7,4245,86_r1_L7,47.412226,5.632973,3,TGCGCACCCTTAGCGAGAGGTTTATCATTAAGGTCAACCTCTGGAT...,5.567218,139,-0.522093
item_16,r1,L7,4250,86_r1_L7,55.899669,13.984365,3,TGCGCACCCTTAGCGAGAGGTTTATCATTAAGGTCAACCTCTGGAT...,5.804794,149,-0.445592
item_28,r1,L7,4262,86_r1_L7,62.126424,1.624409,3,TGCGCACCCTTAGCGAGAGGTTTATCATTAAGGTCAACCTCTGGAT...,5.957158,173,-0.403955
item_44,r1,L10,5278,86_r1_L10,111.101818,5.293149,3,CGCACCCTTAGCGAGAGGTTTATCATTAAGGTCAACCTCTGGATGT...,6.795752,148,-0.108121


In [9]:
if torch.cuda.is_available():
    device = torch.device("cuda") 
else:
    device = torch.device("cpu")

seq_length=200

model = retDNNModel(max_len=seq_length,
             conv1kc=64, conv1ks=10, conv1st=1, conv1pd=10, pool1ks=10, pool1st=5 , pdrop1=0.1,#conv_block_1 parameters
             conv2kc=16, conv2ks=5,  conv2st=1, conv2pd=5,  pool2ks=5,  pool2st=5 , pdrop2=0.1, #conv_block_2 parameters
             convdc =2, convdkc=8 , convdks=3 , #dilation block parameters
             fchidden =32, pdropfc=0.25, final=1, #fully connected parameters

             opt="Adam", loss="mse", lr=0.0036797, momentum=0.9, weight_decay=1e-3
             )
model.compile(device=device)

In [13]:
model.load_state_dict(torch.load('models/best_tuned_model.pt', map_location=torch.device('cpu')))

<All keys matched successfully>

In [21]:
import scipy.stats as stats

test_data_loader = DataLoader(retDataset(df_test.index.tolist(), score_dict, seq_dict, max_len=seq_length), batch_size=32, shuffle=False, drop_last=False)

model.eval()
target_list = []
pred_list = []
seq_id_list = []
with torch.no_grad():
    for batch_idx, batch in enumerate(test_data_loader):
        seqid,seq_X,y = batch
        seq_X = seq_X.to(device)
        y = y.to(device)
        out = model(seq_X)
        pred_list.append(out[:,0].to('cpu').detach().numpy())
        target_list.append(y.to('cpu').detach().numpy())
        seq_id_list.append(seqid)
    targets = np.concatenate(target_list)
    preds = np.concatenate(pred_list)
    seq_ids = np.concatenate(seq_id_list)

print(stats.pearsonr(targets[:,0], preds), stats.spearmanr(targets[:,0], preds))


(0.6528505556116058, 6.410397859585574e-43) SpearmanrResult(correlation=0.6902717241268644, pvalue=1.0347411471838575e-49)
